In [211]:
import mne

In [212]:
import pandas as pd

In [213]:
import json

In [214]:
# %matplotlib inline
import matplotlib.pyplot as plt

In [215]:
import numpy as np

In [216]:
import matplotlib

In [217]:
%load_ext autoreload

%autoreload 2
import tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [218]:
import datetime as dt

In [219]:
import eeg_events

# Чтение сигнала и событий

In [220]:
ch_names = ["Fp1", "Fp2", "C3", "C4", "P7", "P8", 'O1', 'O2']

In [221]:
sampling_freq = 256
n_channels = len(ch_names)
ch_types = ["eeg"] * n_channels

In [222]:
events_name = f"/home/anatoli/Документы/skywind/eeg-playground/sasha-data/actions/casino-bci-sasha-6.json"

In [223]:
baseline_name = f"/home/anatoli/Документы/skywind/eeg-playground/sasha-data/eeg/sasha6.txt"

In [224]:
events_df = tools.read_events_df(events_name, lambda x: True)

In [226]:
baseline_raw = tools.read_signal(
    baseline_name, 
    ch_names=ch_names, 
    ch_types=ch_types, 
    sampling_freq=sampling_freq
)

/home/anatoli/Документы/skywind/eeg-playground/tools.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  sep=", ", skiprows=6, names=["#"] + ch_names + ["x", "y", "z", "time", "timestamp"]


Creating RawArray with float64 data, n_channels=8, n_times=108904
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Setting up band-stop filter from 48 - 52 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 48.00
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 47.75 Hz)
- Upper passband edge: 52.00 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 52.25 Hz)
- Filter length: 1691 samples (6.605 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lo

# Фильтрация

In [1]:
baseline_raw

NameError: name 'baseline_raw' is not defined

In [2]:
# time_stats = (75, 30)
time_stats = (
    15, 
    400
)

In [312]:
raw_part = baseline_raw.copy() \
    .crop(tmin=time_stats[0], tmax=time_stats[1])


In [314]:
tools.plot_frequencies(raw_part, frequencies=(1, 50), start=0, duration=20)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 845 samples (3.301 sec)



In [315]:
ica = mne.preprocessing.ICA()

In [316]:
ica.fit(raw_part.copy())

Fitting ICA to data using 8 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Selecting all PCA components: 8 components
Fitting ICA took 0.9s.


<ICA | raw data decomposition, fit (fastica): 98561 samples, 8 components, channels used: "eeg">

In [317]:
ica_eeg = ica.get_sources(raw_part)

In [318]:
_ = mne.viz.plot_raw(
    ica_eeg,
    duration=30, 
    start=0, 
    scalings=5,
    show=True
)

In [322]:
transformed_raw = ica.apply(raw_part.copy(), exclude=[0, 5]).pick(["Fp1", "Fp2", "O1", "O2"])

Transforming to ICA space (8 components)
Zeroing out 2 ICA components


In [323]:
tools.plot_frequencies(transformed_raw, frequencies=(1, 50), start=0, duration=20)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 845 samples (3.301 sec)



# Сохранение

In [324]:
raw_alpha = tools.get_alpha_amplitude(transformed_raw, sampling_freq=sampling_freq, channels=["O1", "O2"])#.crop(tmin=time_stats[0], tmax=time_stats[1])

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 423 samples (1.652 sec)

Creating RawArray with float64 data, n_channels=5, n_times=98561
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.


In [326]:
_ = mne.viz.plot_raw(
    raw_alpha,
    duration=30, 
    start=0, 
    scalings={
        "eeg": 100,
        "misc": 1
    },
    show=True
)

In [327]:
eeg_events.save_events(raw_alpha, events_df, 'test-filtered')

Writing /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif
Closing /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif [done]


In [238]:
# it = eeg_events.iterate_events('test1')